In [1]:
import pandas as pd
import sys
import math
import imp
import pymongo as pym
import json
import numpy as np
import random
import sys
sys.path.insert(0, './lib_python/')

In [2]:
urlMongoDbLocal = "mongodb://localhost:27017";
clientLocal = pym.MongoClient(urlMongoDbLocal)

In [3]:
carSharing = clientLocal["bigDive_test"]
bookingsDB = carSharing["bookings"]
hexs = carSharing["hexs_gen"]

In [6]:
import libHex
from shapely.geometry import shape
from shapely.geometry import mapping
from shapely.geometry import MultiPolygon
import folium
import seaborn as sns
from shapely.ops import cascaded_union
import json

#name_p_start = "origin_point"
#name_p_end = "dest_point"

def compute_direction_fast(hexs_start, percent = 10, hourInter = [0,24], 
                           days=[1,1,1,1,1,1,1], fluxes = "end"):
    
    name_p_start = "origin_point"
    name_p_end = "dest_point" 
    if fluxes == "start":
        name_point = name_p_start
        name_point2 = name_p_end
    elif fluxes == "end":
        name_point2 = name_p_start
        name_point = name_p_end
    else:
        print("error 'fluxes' must be equals to 'end' or 'start'.")
        return False

    area_union = libHex.unionHexs(hexs_start)
    center = [0, 0]
    counter = [0, 0]
    for i_pol_pol, pol_pol in enumerate(area_union["coordinates"]):
        for i_pol, pol in enumerate(area_union["coordinates"][i_pol_pol]):
            area_union["coordinates"][i_pol_pol][i_pol].append(area_union["coordinates"][i_pol_pol][i_pol][0])
            center[0] += np.sum([x[1] for x in area_union["coordinates"][i_pol_pol][i_pol]])
            center[1] += np.sum([x[0] for x in area_union["coordinates"][i_pol_pol][i_pol]])
            counter[0] += len([x[1] for x in area_union["coordinates"][i_pol_pol][i_pol]])
            counter[1] += len([x[0] for x in area_union["coordinates"][i_pol_pol][i_pol]])
    center[0] = center[0] / counter[0] 
    center[1] = center[1] / counter[1] 

    count = 0
    lat_mean_end = 0
    lon_mean_end = 0
    num_mean_end = 0
    num_dir = {"pp": 0,
    "pm" : 0,
    "mm" : 0,
    "mp" : 0}
    end_dir = {"pp": [0,0],
    "pm" : [0,0],
    "mm" : [0,0],
    "mp" : [0,0]}
    findObj = {}
    findObj[name_point] = {
                "$geoWithin":{
                    "$geometry":area_union
                }
            }
    for trip in bookingsDB.find(findObj):
        lat = trip[name_point2]["coordinates"][1]
        lon = trip[name_point2]["coordinates"][0]
        if (lat > center[0]) and  (lon >= center[1]):
            num_dir["pp"] += 1
            end_dir["pp"][0] += lat
            end_dir["pp"][1] += lon
        if (lat <= center[0]) and (lon > center[1]):
            num_dir["mp"] += 1
            end_dir["mp"][0] += lat
            end_dir["mp"][1] += lon
        if (lat < center[0]) and (lon <= center[1]):
            num_dir["mm"] += 1
            end_dir["mm"][0] += lat
            end_dir["mm"][1] += lon
        if (lat >= center[0]) and (lon < center[1]):
            num_dir["pm"] += 1
            end_dir["pm"][0] += lat
            end_dir["pm"][1] += lon
            
        num_mean_end += 1
        lat_mean_end += trip[name_point2]["coordinates"][1]
        lon_mean_end += trip[name_point2]["coordinates"][0]
        #print("max {0}, count {1}".format(max_,count), end="\r")
    if num_mean_end > 0:
        end_p = [lat_mean_end / num_mean_end, lon_mean_end / num_mean_end]
        diff = [(end_p[0] - center[0]) * percent / 100, (end_p[1] - center[1]) * percent / 100]
        end_p = [center[0] + diff[0], center[1] + diff[1]]
    else:
        end_p = center
    for pp in end_dir:
        if num_dir[pp] > 0:
            end_dir[pp] = [end_dir[pp][0] / num_dir[pp], end_dir[pp][1] / num_dir[pp]]
            diff = [(end_dir[pp][0] - center[0]) * percent / 100, (end_dir[pp][1] - center[1]) * percent / 100]
            end_dir[pp] = [center[0] + diff[0], center[1] + diff[1]]
        else:
            end_dir[pp] = center
    return {"center_p" : center, "end_p" : end_p, "num_mean_end" : num_mean_end, "num_dir":num_dir, "end_dir": end_dir}



In [7]:
count = 0
city = "Milano"
center = hexs.find_one({"city":city})["point"]["coordinates"]
map_folium =  folium.Map(location = [ center[1], center[0]], zoom_start=12,  tiles="Stamen Toner")
for p in hexs.find({"city":city}):
    locs = compute_direction_fast([p], fluxes = "start")
    weight = np.log(locs["num_mean_end"]) if locs["num_mean_end"] else 0
    weight = locs["num_mean_end"]
    folium.PolyLine(locations=[locs["center_p"],locs["end_p"]]).add_to(map_folium)
    folium.Circle(location=locs["center_p"], color = "red",radius = weight/10).add_to(map_folium)

    print(count,weight, end="\r")
    count += 1
    if count % 50 == 0:
        map_folium.save("map_fluxes.html")
map_folium

In [13]:
city = "Milano"
center = hexs.find_one({"city":city})["point"]["coordinates"]
map_folium =  folium.Map(location = [ center[1], center[0]], zoom_start=12,  tiles="Stamen Toner")
count = 0
for p in hexs.find():
    locs = compute_direction_fast([p], percent=5,  fluxes = "end")
    locs_start = compute_direction_fast([p], percent=5, fluxes = "start")
    weight_in = locs["num_mean_end"]/10
    weight_out = locs_start["num_mean_end"]/10
    color = "green" if (weight_in < weight_out) else "red"
    if (weight_in == weight_out):
        color = "grey"
    folium.PolyLine(locations=[locs_start["center_p"],locs_start["end_p"]], color = color, ).add_to(map_folium)
    if color == "red":
        folium.Circle(location=locs["center_p"], color = "red", fill_color = "red",fill = True,fill_opacity = 1, radius = weight_in, ).add_to(map_folium)
        #folium.Circle(location=locs["center_p"], color = "green", fill_color = "green",fill_opacity = 1,fill = True,radius = weight_out).add_to(map_folium)
    else:
        folium.Circle(location=locs["center_p"], color = "green", fill_color = "green", fill_opacity = 1,fill = True,radius = weight_out).add_to(map_folium)
        #folium.Circle(location=locs["center_p"], color = "red", fill_color = "red",fill_opacity = 1, fill = True,radius = weight_in, ).add_to(map_folium)

    print(count,weight, end="\r")
    count += 1
    if count % 50 == 0:
        map_folium.save("map_holes_sources.html")
map_folium